# The Bachelor

In this notebook I will dive into the data from the show The Bachelor. Lets get it.

In [1]:
%pip install --upgrade plotly pandas numpy plotly seaborn matplotlib nbformat


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Load the packages

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

### Format the Notebook

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the dataset

In [4]:
df_bachelor = pd.read_csv('historical_bachelor_contestants.csv')

### The initial look at the data set

In [5]:
print(df_bachelor.shape)
print(df_bachelor.columns)
print(df_bachelor.dtypes)

(479, 8)
Index(['Unnamed: 0', 'Age', 'Eliminated', 'Hometown', 'Name', 'Occupation',
       'Outcome', 'Season'],
      dtype='object')
Unnamed: 0      int64
Age           float64
Eliminated     object
Hometown       object
Name           object
Occupation     object
Outcome        object
Season          int64
dtype: object


In [6]:
print(df_bachelor.isna().any())

Unnamed: 0    False
Age            True
Eliminated     True
Hometown      False
Name          False
Occupation    False
Outcome        True
Season        False
dtype: bool


In [7]:
df_bachelor.head()

,Unnamed: 0,Age,Eliminated,Hometown,Name,Occupation,Outcome,Season
0,0,23.00,Winner,"Chanute, Kansas",Amanda Marsh,Event Planner,NaN,1
1,1,29.00,Runner-Up,"Miami, Florida",Trista Rehn,Miami Heat Dancer,NaN,1
2,2,24.00,Week 5,"Dallas, Texas",Shannon Oliver,Financial Management Consultant,NaN,1
3,3,24.00,Week 4,"Tempe, Arizona",Kim,Nanny,NaN,1
4,4,22.00,Week 3,"Terra Haute, Indiana",Cathy Grimes,Graduate Student,NaN,1


In [8]:
winners_elem = df_bachelor.query('Eliminated == "Winner"')

In [9]:
win_outcome = df_bachelor.query('Outcome == "Winner"')

In [15]:
df_winners = pd.concat([winners_elem, win_outcome])
df_winners

,Unnamed: 0,Age,Eliminated,Hometown,Name,Occupation,Outcome,Season
0,0,23.00,Winner,"Chanute, Kansas",Amanda Marsh,Event Planner,NaN,1
25,0,27.00,Winner,"Gloucester, New Jersey",Helene Eksterowicz,School Psychologist,NaN,2
50,0,22.00,Winner,"Huntington Beach, California",Jessica Bowlin,Student,NaN,5
75,0,24.00,Winner,"Pembroke Pines, Florida",Jennifer Wilson,Teacher,NaN,9
102,0,26.00,Winner,"San Francisco, California",Tessa Horst,Social Worker,NaN,10
152,0,22.00,NaN,"Malibu, California",Shayne Lamas[1],Actress,Winner,12
177,0,25.00,NaN,"Dallas, Texas",Melissa Rycroft[1],Sales Representative,Winner,13
202,0,23.00,NaN,"Geneva, Florida",Vienna Girardi[1][2],Marketing Representative,Winner,14
227,0,25.00,NaN,"Morgantown, West Virginia",Emily Maynard[5],Children's Hospital Event Planner,Winner,15
257,0,28.00,NaN,"Scottsdale, Arizona",Courtney Robertson[3][4],Model,Winner,16


In [16]:
df_winners.drop(['Eliminated', 'Outcome', 'Unnamed: 0'],axis=1, inplace=True)
df_winners

,Age,Hometown,Name,Occupation,Season
0,23.00,"Chanute, Kansas",Amanda Marsh,Event Planner,1
25,27.00,"Gloucester, New Jersey",Helene Eksterowicz,School Psychologist,2
50,22.00,"Huntington Beach, California",Jessica Bowlin,Student,5
75,24.00,"Pembroke Pines, Florida",Jennifer Wilson,Teacher,9
102,26.00,"San Francisco, California",Tessa Horst,Social Worker,10
152,22.00,"Malibu, California",Shayne Lamas[1],Actress,12
177,25.00,"Dallas, Texas",Melissa Rycroft[1],Sales Representative,13
202,23.00,"Geneva, Florida",Vienna Girardi[1][2],Marketing Representative,14
227,25.00,"Morgantown, West Virginia",Emily Maynard[5],Children's Hospital Event Planner,15
257,28.00,"Scottsdale, Arizona",Courtney Robertson[3][4],Model,16


In [17]:
df_winners.columns

Index(['Age', 'Hometown', 'Name', 'Occupation', 'Season'], dtype='object')

In [30]:
df_winners.sort_values(by='Age')
age_count = df_winners.groupby('Age', as_index=False).agg({'Season' : pd.Series.count})
age_count.columns = ['Age', 'age-total']
age_count

,Age,age-total
0,22.00,2
1,23.00,2
2,24.00,1
3,25.00,3
4,26.00,3
5,27.00,2
6,28.00,1
7,29.00,1


In [31]:
fig = px.bar(age_count, x=age_count['Age'], y=age_count['age-total'])
fig.show()